In [1]:
from pinecone import Pinecone as PineconeClient
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda


In [2]:
import os
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = "aws"
PINECONE_INDEX_NAME ="motionpicturedotorg"

In [3]:
pinecone = PineconeClient(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

In [4]:
embeddings = OpenAIEmbeddings()
vectorstore = Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME, embedding=embeddings)
retriever = vectorstore.as_retriever()

## RAG Prompt

In [5]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [6]:
chain.invoke("what does New Mexico offer to movie production companies?")

'New Mexico offers a favorable tax credit, relatively low living and production costs, and is considered a film-friendly state. The state has a supportive legislature and a highly regarded New Mexico Film Office (NMFO) team. Additionally, the NMFO has provided guidance and resources such as protective equipment and testing to help the film and television sector safely return to work following COVID-19 disruptions.'

In [7]:
chain.invoke("what are the opportunities to film in New Mexico?")

'The opportunities for filming in New Mexico, as mentioned in the provided context, include:\n\n1. New Mexico being regarded as film-friendly by many consultees.\n2. Relatively low living and production costs.\n3. A favorable tax credit system for film production.\n\nThese factors contribute to making New Mexico an attractive location for film and television production.'